In [1]:
!pip install mwxml
!pip install mwparserfromhell
!pip install spacy
!pip install langchain
!pip install pandas
!pip install qdrant-client
!pip install langchain-openai tiktoken
!pip install sqlalchemy==2.0.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant

import getpass
import pandas as pd
import os
import csv
import mwxml
import mwparserfromhell
from tqdm import tqdm
from spacy.lang.en import English
from urllib.parse import unquote

In [3]:
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()
qdrant_api_key = getpass.getpass()

········
········


In [4]:
DUMP_PATH = "scratch/enwiki-20240120-pages-articles-multistream.xml"
WIKIDATA_FILM_ENTITIES = "scratch/wikidata/english_dump.nt"
OUTPUT_CSV_PATH="scratch/wikidata/film_documents.csv"
AUTOCOMPLETE_OUTPUT_PATH="scratch/wikidata/autocomplete_phrases.txt"
TITLES_INV_INDEX_PATH="scratch/wikidata/id_title.csv"

In [10]:
def get_film_title(dump_line: str):
    """
    dump_line = "<https://en.wikipedia.org/wiki/Prometheus_%282012_film%29>"
    """
    return unquote(line.split("https://en.wikipedia.org/wiki/")[-1].split(">")[0]).replace("_", " ")

def strip_code_from_mwtext(text: str):
    return mwparserfromhell.parse(revision.text).strip_code(normalize=True, collapse=True, keep_template_params=False)

In [13]:
# Build set of pages we want
titles = set()

with open(WIKIDATA_FILM_ENTITIES) as fp:
    for line in fp.readlines():
        titles.add(get_film_title(line.replace("\n", "")))

total_titles = len(titles)
print (len(titles))        

149383


In [ ]:
def extract_revisions_of_interest(output_path: str):
    fp = open(output_path, "w")
    output = csv.writer(fp, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    output.writerow(["id", "title", "text"])
    dump = mwxml.Dump.from_file(open(DUMP_PATH))
    site_metadata = dump.site_info

    hits = 0
    misses = 0
    pbar = tqdm(dump)
    for page in pbar:
        pbar.set_description(f"Hits: {hits} / {total_titles}")
        row = [page.id]
        if page.title in titles:
            hits += 1
            for revision in page:
                code = mwparserfromhell.parse(revision.text)
                row.append(page.title)
                text = '\n\n'.join([s.strip_code(normalize=True, collapse=True, keep_template_params=False) for s in code.get_sections()[:2]])
                row.append(text)
                output.writerow(row)
                break
        else:
            misses += 1

    print(f"Processed {hits + misses} pages: {hits} hits, {misses} misses")

extract_revisions_of_interest(OUTPUT_CSV_PATH)

Hits: 48 / 149383: : 12039it [00:15, 1015.19it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 105 / 149383: : 24544it [00:32, 1194.66it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 161 / 149383: : 33286it [00:42, 914.35it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg

Hits: 1087 / 149383: : 144640it [02:41, 978.15it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 1139 / 149383: : 151104it [02:48, 905.99it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 1161 / 149383: : 154927it [02:51, 1225.53it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.io

Hits: 1557 / 149383: : 237239it [04:02, 1315.38it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 1570 / 149383: : 241423it [04:05, 1321.00it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 1585 / 149383: : 245426it [04:09, 1308.64it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.io

Hits: 1949 / 149383: : 327836it [05:15, 1493.04it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 1969 / 149383: : 332042it [05:18, 1567.69it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 1991 / 149383: : 336057it [05:22, 1270.18it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.io

Hits: 2434 / 149383: : 418358it [06:27, 1342.13it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 2463 / 149383: : 422325it [06:31, 1530.17it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 2482 / 149383: : 428627it [06:35, 1179.35it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.io

...

Hits: 148188 / 149383: : 22711159it [4:03:36, 1638.90it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 148217 / 149383: : 22717275it [4:03:39, 1780.50it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 148245 / 149383: : 22722360it [4:03:43, 1579.04it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable

Hits: 148964 / 149383: : 22838983it [4:04:53, 1820.90it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 149003 / 149383: : 22844966it [4:04:57, 1552.71it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 149047 / 149383: : 22850546it [4:05:00, 1666.21it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable

Hits: 149764 / 149383: : 22966767it [4:06:11, 1609.50it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 149792 / 149383: : 22972622it [4:06:15, 1631.30it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 149822 / 149383: : 22978187it [4:06:18, 1620.43it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable

Hits: 150586 / 149383: : 23094783it [4:07:29, 1709.87it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 150636 / 149383: : 23100944it [4:07:33, 1652.74it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 150684 / 149383: : 23106434it [4:07:36, 1596.00it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable

Hits: 151468 / 149383: : 23209353it [4:08:46, 1629.30it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 151533 / 149383: : 23216800it [4:08:51, 1563.83it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Hits: 151590 / 149383: : 23222606it [4:08:54, 1616.23it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable

## Write the titles out for autocomplete

In [5]:
def write_auto_complete_file():
    df = pd.read_csv(OUTPUT_CSV_PATH, usecols=["title"], delimiter=",", quotechar='|')
    df.to_csv(AUTOCOMPLETE_OUTPUT_PATH, index=False, header=False)

write_auto_complete_file()

## Write a dataset to build inverted index on the titles

In [6]:
def write_titles_index_dataset():
    df = pd.read_csv(OUTPUT_CSV_PATH, usecols=["id", "title"], delimiter=",", quotechar='|')
    df.to_csv(TITLES_INV_INDEX_PATH, index=False, header=True, quotechar='|')

write_titles_index_dataset()

## Index the new dataset using Embeddings and Vector DB

In [7]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant

def load_and_split_docs():
    text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=0)

    loader = CSVLoader(
        file_path=OUTPUT_CSV_PATH,
        source_column="id",
        csv_args={
            "delimiter": ",",
            "quotechar": '|',
            "fieldnames": ["id", "title", "text"],
        }
    )

    print("Loading documents")
    documents = loader.load()
    docs = text_splitter.split_documents(documents)
    print("Documents loaded")
    
    return docs


def index():
    print("Building index")
    qdrant = Qdrant.from_documents(
        docs,
        embeddings,
        url="https://3faefa0b-c722-44af-bb0f-c6fbf173c96b.us-east4-0.gcp.cloud.qdrant.io",
        prefer_grpc=True,
        api_key=qdrant_api_key,
        collection_name="deeno",
        force_recreate=True
    )
    print("Index built")

docs = load_and_split_docs()
embeddings = OpenAIEmbeddings()
index()

Loading documents


Created a chunk of size 4544, which is longer than the specified 3000
Created a chunk of size 7175, which is longer than the specified 3000
Created a chunk of size 5290, which is longer than the specified 3000
Created a chunk of size 3276, which is longer than the specified 3000
Created a chunk of size 3781, which is longer than the specified 3000
Created a chunk of size 6718, which is longer than the specified 3000
Created a chunk of size 3062, which is longer than the specified 3000
Created a chunk of size 17707, which is longer than the specified 3000
Created a chunk of size 4922, which is longer than the specified 3000
Created a chunk of size 3643, which is longer than the specified 3000
Created a chunk of size 7055, which is longer than the specified 3000
Created a chunk of size 3226, which is longer than the specified 3000
Created a chunk of size 6864, which is longer than the specified 3000
Created a chunk of size 3977, which is longer than the specified 3000
Created a chunk of 

Documents loaded


/home/hice1/skumar749/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Building index


KeyboardInterrupt: 